## Interação Inteligente para PDI de tecnologias para o CEIS/SUS

O spaCy separa os modelos de linguagem dos seus componentes principais para permitir uma instalação mais flexível e eficiente. Após a instalação da biblioteca SpaCy no seu ambiente atual, para instalar os modelos específicos que deseja usar para cada idioma executar o comando a seguir para baixar e instalar o modelo 'pt_core_news_lg' no seu ambiente atual:

    python -m spacy download pt_core_news_lg 

O spaCy oferece outros modelos em português, como o 'pt_core_news_sm' (menor e mais rápido) e modelos treinados com transformers (pt_core_news_trf). Você pode explorar esses modelos se precisar de diferentes recursos ou tamanhos.

In [7]:
# %pip install neo4j
# %pip show spacy
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 11.4 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [8]:
!python -m spacy download pt_core_news_trf


✘ No compatible package found for 'pt_core_news_trf' (spaCy v3.7.5)



In [9]:
import inspect

def mapear_elementos_modulo(nome_arquivo):
    """
    Mapeia e imprime as classes, métodos e funções dentro de um arquivo Python.

    Args:
        nome_arquivo: O nome do arquivo .py a ser analisado.
    """

    # Importar o módulo dinamicamente
    modulo = __import__(nome_arquivo.replace(".py", ""))

    # Obter todos os membros do módulo
    membros = inspect.getmembers(modulo)

    # Filtrar e imprimir classes, métodos e funções
    for nome, elemento in membros:
        if inspect.isclass(elemento):
            print(f"Classe: {nome}")
            for nome_metodo, metodo in inspect.getmembers(elemento, inspect.isfunction):
                if nome_metodo.startswith("__"):  # Ignorar métodos especiais
                    continue
                print(f"  Método: {nome_metodo}")
        elif inspect.isfunction(elemento):
            if nome.startswith("__"):  # Ignorar funções especiais
                continue
            print(f"Função: {nome}")

In [11]:
# Exemplo de uso
nome_arquivo = "analisar_pdi_ceis_sus_grafo.py"
mapear_elementos_modulo(nome_arquivo)

Classe: AnaliseGrafo
  Método: analisar_distribuicao_graus
  Método: calcular_densidade_grafo
  Método: visualizar_distribuicao_graus
Classe: ExplosaoSubgrafos
  Método: apresentar_subgrafo
  Método: recuperar_subgrafo
Classe: FontesDados
  Método: consultar_fontes
Classe: GraphDatabase
Classe: InteracaoUsuario
  Método: gerar_consulta_neo4j
  Método: interagir_com_pesquisador
Classe: MapeamentoEntidades
  Método: mapear_entidades
  Método: mapear_entidades_gpu
  Método: sugerir_novas_entidades
  Método: sugerir_novas_entidades_gpu
Classe: ProcessamentoLinguagemNatural
  Método: processar_questao_pesquisa
Classe: RecomendacaoProjetos
  Método: recomendar_projetos
Classe: SentenceTransformer
  Método: _apply
  Método: _call_impl
  Método: _create_model_card
  Método: _encode_multi_process_worker
  Método: _eval_during_training
  Método: _first_module
  Método: _get_backward_hooks
  Método: _get_backward_pre_hooks
  Método: _get_item_by_idx
  Método: _get_name
  Método: _get_scheduler
  

# Extrair entidades e relacionamentos de documentos

In [ ]:
import json
import spacy
from tika import parser  # Para extrair texto de PDFs
from bs4 import BeautifulSoup  # Para analisar HTML
from neo4j import GraphDatabase

# Conectar ao banco de dados Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Carregar o JSON
with open("dados_pesquisa_biotecnologia.json", "r") as f:
    data = json.load(f)

# Criar nós e relacionamentos no Neo4j
with driver.session() as session:
    for node in data["nodes"]:
        session.run(
            "CREATE (n:%s {name: $name, legislação: $legislação})" % node["label"],
            **node["properties"]
        )

    for relationship in data["relationships"]:
        session.run(
            "MATCH (s {name: $source_name}), (t {name: $target_name}) "
            "CREATE (s)-[:%s]->(t)" % relationship["type"],
            source_name=relationship["source"],
            target_name=relationship["target"]
        )

driver.close()

# Carregar o modelo de linguagem português do spaCy (certificar antes que o modelo já está instalado para funcionar)
nlp = spacy.load("pt_core_news_lg") 

def extrair_informacoes_documento(caminho_documento):
    """
    Extrai informações de um documento em HTML ou PDF e retorna um dicionário com entidades e relacionamentos.
    Limitado inicialmente no MVP para sujeitos e objetos diretos mediados por verbos
    """

    # Extrair o texto do documento (adapte para outros formatos se necessário)
    if caminho_documento.endswith(".pdf"):
        raw_text = parser.from_file(caminho_documento)["content"]
    elif caminho_documento.endswith(".html"):
        with open(caminho_documento, "r", encoding="utf-8") as f:
            html_content = f.read()
        soup = BeautifulSoup(html_content, "html.parser")
        raw_text = soup.get_text()
    else:
        raise ValueError("Formato de arquivo não suportado")

    # Processar o texto com spaCy
    doc = nlp(raw_text)

    # Extrair entidades e relacionamentos (adaptar regras de acordo com o tipo de documento)
    entidades = []
    relacionamentos = []

    for sent in doc.sents:
        # Identificar possíveis atividades e documentos
        for token in sent:
            if token.pos_ == "NOUN" and token.dep_ in ["nsubj", "dobj"]:  # sujeito ou objeto direto
                entidades.append({"id": token.text, "label": "Atividade" if token.dep_ == "nsubj" else "Documento"})

        # Identificar possíveis relacionamentos entre atividades e documentos
        for token in sent:
            if token.dep_ == "ROOT" and token.head.pos_ == "VERB":  # verbo principal da frase
                sujeito = [child for child in token.children if child.dep_ == "nsubj"]
                objeto = [child for child in token.children if child.dep_ == "dobj"]
                if sujeito and objeto:
                    relacionamentos.append({
                        "source": sujeito[0].text,
                        "target": objeto[0].text,
                        "type": "REQUER"  # Adapte o tipo de relacionamento conforme necessário
                    })

    # Construir o JSON de saída
    dados_json = {
        "nodes": entidades,
        "relationships": relacionamentos
    }

    return dados_json

In [ ]:
# Exemplo de uso
caminho_documento = "caminho/para/seu/documento.pdf"  # ou .html
dados_extraidos = extrair_informacoes_documento(caminho_documento)
dados_extraidos

In [ ]:
# Salvar o JSON em um arquivo
with open("dados_extraidos.json", "w", encoding="utf-8") as f:
    json.dump(dados_extraidos, f, ensure_ascii=False, indent=4)

# Avaliar Similaridades Semânticas por métricas diversas

O Neo4j oferece algoritmos de similaridade diretamente em suas consultas Cypher, o que pode ser mais eficiente em alguns casos do que calcular as similaridades em Python e depois ordená-las. 

A escolha da métrica de similaridade ideal dependerá do seu caso de uso específico e das características dos seus dados. É fundamental entender as particularidades de cada métrica para interpretar corretamente os resultados.

In [ ]:
import json
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer, util # Para calcular similaridade semântica

# Conectar ao banco de dados Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Carregar o modelo de similaridade semântica (escolha um modelo adequado)
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 

def calcular_similaridade_euclidiana(caminho_json):
    """
    Calcula a similaridade euclidiana entre entidades em um arquivo JSON local e entidades
    existentes no banco de dados Neo4j, retornando um índice ordenado de similaridade.
    """

    # Carregar o JSON local
    with open(caminho_json, "r", encoding="utf-8") as f:
        dados_json = json.load(f)

    # Extrair entidades do JSON local
    entidades_locais = [node["properties"]["name"] for node in dados_json["nodes"]]

    # Consultar entidades e seus embeddings no Neo4j (assumindo que você já armazenou os embeddings)
    with driver.session() as session:
        resultado = session.run("MATCH (n) RETURN n.name AS name, n.embedding AS embedding")
        entidades_neo4j = [(registro["name"], registro["embedding"]) for registro in resultado]

    # Calcular similaridade euclidiana usando Cypher
    indice_ordenado = []
    for entidade_local in entidades_locais:
        embedding_local = model.encode(entidade_local, convert_to_tensor=True).tolist()[0]  # Converter para lista
        query = f"""
            WITH point({embedding_local}) AS p1
            UNWIND $embeddings AS p2
            RETURN p2.name AS name, distance(p1, point(p2.embedding)) AS distance
            ORDER BY distance
        """
        params = {"embeddings": entidades_neo4j}
        resultado = session.run(query, params)

        for registro in resultado:
            indice_ordenado.append({
                "entidade_local": entidade_local,
                "entidade_neo4j": registro["name"],
                "similaridade": 1 / (1 + registro["distance"])  # Converter distância em similaridade
            })

    indice_ordenado.sort(key=lambda x: x["similaridade"], reverse=True)

    return indice_ordenado

def calcular_similaridade_jaccard(caminho_json):
    """
    Calcula a similaridade de Jaccard entre entidades em um arquivo JSON local e entidades
    existentes no banco de dados Neo4j, retornando um índice ordenado de similaridade.
    """

    # Carregar o JSON local
    with open(caminho_json, "r", encoding="utf-8") as f:
        dados_json = json.load(f)

    # Extrair entidades do JSON local
    entidades_locais = [node["properties"]["name"] for node in dados_json["nodes"]]

    # Consultar entidades e seus embeddings no Neo4j (assumindo que você já armazenou os embeddings)
    with driver.session() as session:
        resultado = session.run("MATCH (n) RETURN n.name AS name, n.embedding AS embedding")
        entidades_neo4j = [(registro["name"], registro["embedding"]) for registro in resultado]

    # Calcular similaridade de Jaccard usando Cypher (requer converter embeddings para strings)
    indice_ordenado = []
    for entidade_local in entidades_locais:
        embedding_local_str = ','.join(map(str, model.encode(entidade_local, convert_to_tensor=True).tolist()[0]))
        query = f"""
            WITH split($embedding_local, ',') AS s1
            UNWIND $embeddings AS e2
            WITH s1, e2, split(e2.embedding, ',') AS s2
            RETURN e2.name AS name, 
                   toFloat(size(apoc.coll.intersection(s1, s2))) / toFloat(size(apoc.coll.union(s1, s2))) AS similarity
            ORDER BY similarity DESC
        """
        params = {"embedding_local": embedding_local_str, "embeddings": entidades_neo4j}
        resultado = session.run(query, params)

        for registro in resultado:
            indice_ordenado.append({
                "entidade_local": entidade_local,
                "entidade_neo4j": registro["name"],
                "similaridade": registro["similarity"]
            })

    return indice_ordenado

def calcular_similaridade_entidades(caminho_json, algoritmo_similaridade="cosine"):
    """
    Calcula, por um algoritmo definido, a similaridade semântica entre entidades em um arquivo JSON local e entidades
    existentes no banco de dados Neo4j, retornando um índice ordenado de similaridade.
    """

    # Carregar o JSON local
    with open(caminho_json, "r", encoding="utf-8") as f:
        dados_json = json.load(f)

    # Extrair entidades do JSON local
    entidades_locais = [node["properties"]["name"] for node in dados_json["nodes"]]

    # Consultar entidades existentes no Neo4j
    with driver.session() as session:
        resultado = session.run("MATCH (n) RETURN n.name AS name")
        entidades_neo4j = [registro["name"] for registro in resultado]

    # Calcular embeddings das entidades (representações vetoriais)
    embeddings_locais = model.encode(entidades_locais, convert_to_tensor=True)
    embeddings_neo4j = model.encode(entidades_neo4j, convert_to_tensor=True)

    # Calcular similaridade (escolha o algoritmo desejado)
    if algoritmo_similaridade == "cosine":
        similaridades = util.cos_sim(embeddings_locais, embeddings_neo4j)
    # Adicione outros algoritmos de similaridade aqui, se necessário

    # Criar um índice ordenado de similaridade
    indice_ordenado = []
    for i, entidade_local in enumerate(entidades_locais):
        for j, entidade_neo4j in enumerate(entidades_neo4j):
            indice_ordenado.append({
                "entidade_local": entidade_local,
                "entidade_neo4j": entidade_neo4j,
                "similaridade": similaridades[i][j].item()  # Converter tensor para valor numérico
            })

    indice_ordenado.sort(key=lambda x: x["similaridade"], reverse=True)

    return indice_ordenado


In [ ]:
# Cálculo por similaridade euclidiana
caminho_json = "caminho/para/seu/arquivo.json"
calcular_similaridade_euclidiana(caminho_json)

In [ ]:
# Cálculo por similaridade de Jaccard
caminho_json = "caminho/para/seu/arquivo.json"
calcular_similaridade_jaccard(caminho_json)

In [ ]:
# Cálculo por similaridade de cossenos
caminho_json = "caminho/para/seu/arquivo.json"
algoritmo = "cosine"  # ou outro algoritmo de similaridade disponível
resultados = calcular_similaridade_entidades(caminho_json, algoritmo)

# Imprimir os resultados
for resultado in resultados:
    print(f"Entidade local: {resultado['entidade_local']}")
    print(f"Entidade Neo4j: {resultado['entidade_neo4j']}")
    print(f"Similaridade: {resultado['similaridade']}\n")

Para recomendar a métrica mais adequada para calcular a similaridade semântica entre entidades e relacionamentos em grafos no Neo4j, precisamos analisar as características dos dados e a topologia dos grafos envolvidos. Para análise genérica inicial, ainda desconsiderando dados e estrutura de algum grafo específicos, utilizamos um método baseado em informações sobre o tipo de dados e a estrutura do grafo inferidos pelo texto da questão de pesquisa fornecida pelo usuário para fazer a recomendação, juntamente com uma breve explicação da escolha.

In [ ]:
from neo4j import GraphDatabase
import networkx as nx  # Para análise de grafos
import numpy as np
import cupy as cp  # Para cálculos em GPU (se disponível)
from collections import Counter
from sklearn.metrics import r2_score

# Conectar ao banco de dados Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "your_password"))

def calcular_densidade_grafo(grafo_neo4j):
    """
    Calcula a densidade do grafo Neo4j.
    """

    with driver.session() as session:
        resultado = session.run("""
            MATCH (n) 
            WITH count(n) AS num_nodes, 
                 sum(size((n)--()))/2 AS num_edges 
            RETURN toFloat(num_edges) / (toFloat(num_nodes) * (toFloat(num_nodes) - 1) / 2) AS density
        """)
        densidade = resultado.single()["density"]

    return densidade

def analisar_distribuicao_graus(grafo_neo4j):
    """
    Analisa a distribuição de graus do grafo Neo4j e retorna "power_law" se a distribuição seguir
    aproximadamente uma lei de potência, ou "other" caso contrário.
    """

    with driver.session() as session:
        resultado = session.run("MATCH (n) RETURN size((n)--()) AS degree")
        graus = [registro["degree"] for registro in resultado]

    # Converter para NumPy array para cálculos mais rápidos
    graus_np = np.array(graus)

    # Opcional: usar CuPy para cálculos em GPU, se disponível
    if cp.cuda.is_available():
        graus_np = cp.asarray(graus_np)

    # Calcular histograma e analisar a distribuição
    histograma, _ = np.histogram(graus_np, bins='auto')

    # Filtrar zeros do histograma e calcular logaritmos
    graus_nao_zero = np.where(histograma > 0)[0]
    log_graus = np.log10(graus_nao_zero + 1)  # Adicionar 1 para evitar log(0)
    log_freqs = np.log10(histograma[graus_nao_zero])

    # Ajustar uma reta aos dados em escala log-log
    coeficientes = np.polyfit(log_graus, log_freqs, 1)

    # Calcular o coeficiente de determinação R²
    y_pred = np.polyval(coeficientes, log_graus)
    r2 = r2_score(log_freqs, y_pred)

    # Definir um limiar para o R² e retornar True ou False
    limiar_r2 = 0.9  # Ajuste conforme necessário
    if r2 >= limiar_r2:
        return "power_law"
    else:
        return "other"

def identificar_tipos_relacionamentos(grafo_neo4j):
    """
    Identifica os tipos de relacionamentos presentes no grafo Neo4j e retorna "homogeneos"
    se houver poucos tipos distintos, ou "heterogeneos" caso contrário.
    """

    with driver.session() as session:
        resultado = session.run("MATCH ()-[r]->() RETURN distinct type(r) AS tipo")
        tipos = [registro["tipo"] for registro in resultado]

    if len(tipos) <= 3:  # Adapte o limiar conforme necessário
        return "homogeneos"
    else:
        return "heterogeneos"

def calcular_tamanho_vocabulario(dados_json, grafo_neo4j):
    """
    Calcula o tamanho do vocabulário combinado das entidades no JSON e no grafo Neo4j.
    """

    # Extrair entidades do JSON local (igual ao exemplo anterior)
    entidades_locais = [node["properties"]["name"] for node in dados_json["nodes"]]

    # Consultar entidades existentes no Neo4j (igual ao exemplo anterior)
    with driver.session() as session:
        resultado = session.run("MATCH (n) RETURN n.name AS name")
        entidades_neo4j = [registro["name"] for registro in resultado]

    # Combinar e contar palavras únicas
    vocabulario = Counter(entidades_locais + entidades_neo4j)
    tamanho_vocabulario = len(vocabulario)

    return tamanho_vocabulario

def recomendar_metrica(dados_json, grafo_neo4j):
    """
    Analisa as características dos dados e do grafo para recomendar a métrica 
    de similaridade semântica mais adequada.

    Args:
        dados_json: Dados no formato JSON a serem comparados com o grafo.
        grafo_neo4j: Objeto representando o grafo no Neo4j.

    Returns:
        Uma tupla contendo a métrica recomendada e uma breve explicação da escolha.
    """

    # Analisar características dos dados e do grafo (código hipotético)
    densidade_grafo = calcular_densidade_grafo(grafo_neo4j)
    distribuicao_graus = analisar_distribuicao_graus(grafo_neo4j)
    tipos_relacionamentos = identificar_tipos_relacionamentos(grafo_neo4j)
    tamanho_vocabulario = calcular_tamanho_vocabulario(dados_json, grafo_neo4j)

    # Lógica de decisão para recomendar a métrica (código hipotético)
    if densidade_grafo > 0.5 and distribuicao_graus == "power_law":
        metrica_recomendada = "cosine"
        explicacao = "Grafo denso com distribuição de graus em lei de potência sugere similaridade baseada em contexto. A distância de cossenos é eficaz em capturar relações semânticas em grafos densos e complexos, onde o contexto das entidades é crucial."
    elif tamanho_vocabulario < 1000 and tipos_relacionamentos == "homogeneos":
        metrica_recomendada = "euclidean"
        explicacao = "Vocabulário pequeno e relacionamentos homogêneos indicam que a distância euclidiana pode ser suficiente. A distância euclidiana é uma métrica simples e eficiente para calcular a similaridade entre vetores em espaços de baixa dimensão, o que pode ser o caso quando o vocabulário é limitado."
    else:
        metrica_recomendada = "jaccard"
        explicacao = "Em outros casos, a similaridade de Jaccard pode ser uma boa opção geral, especialmente quando a presença ou ausência de características é mais importante do que a magnitude das diferenças. A similaridade de Jaccard é útil para comparar conjuntos de elementos, como palavras-chave ou atributos, e é menos sensível a variações na frequência dos termos."

    return metrica_recomendada, explicacao

# Interação com usuários com utilização de PLN

Froam criados métodos em Python que utilizam processamento de linguagem natural (PLN) para interagir com pesquisadores, permitindo que eles expressem suas questões de pesquisa em linguagem natural e recebam sugestões de entidades e relacionamentos relevantes para serem explorados no grafo Neo4j. O sistema também permitirá que os pesquisadores refinem suas consultas, incluindo ou modificando entidades e relacionamentos.

In [ ]:
import spacy
from neo4j import GraphDatabase

# Carregar o modelo de linguagem português do spaCy
nlp = spacy.load("pt_core_news_lg")

# Conectar ao banco de dados Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def processar_questao_pesquisa(questao):
    """
    Processa a questão de pesquisa do usuário utilizando PLN e retorna entidades e 
    relacionamentos relevantes.
    """

    # Analisar a questão com spaCy
    doc = nlp(questao)

    # Extrair entidades e relacionamentos (adapte as regras conforme necessário)
    entidades = [ent.text for ent in doc.ents]
    relacionamentos = [(child.text, child.dep_) for token in doc for child in token.children if child.dep_ in ["nsubj", "dobj"]]

    return entidades, relacionamentos

def gerar_consulta_neo4j(entidades, relacionamentos):
    """
    Gera uma consulta Cypher para o Neo4j com base nas entidades e relacionamentos extraídos.
    """

    # Construir a consulta (adapte a lógica conforme a estrutura do seu grafo)
    match_clause = "MATCH "
    where_clause = "WHERE "
    for entidade in entidades:
        match_clause += f"(n{entidades.index(entidade)}:Entidade {{nome: '{entidade}'}})"
        if entidade != entidades[-1]:
            match_clause += ", "

    for relacionamento in relacionamentos:
        sujeito, tipo_relacionamento = relacionamento
        match_clause += f"-[r{relacionamentos.index(relacionamento)}:{tipo_relacionamento}]->"

    consulta = match_clause + " " + where_clause + " RETURN *"

    return consulta
    
def interagir_com_pesquisador():
    """
    Realiza a interação com o pesquisador, permitindo que ele refine sua consulta.
    """

    entidades = []
    relacionamentos = []

    while True:
        questao = input("Digite sua questão de pesquisa ou 'sair' para encerrar: ")
        if questao.lower() == 'sair':
            break

        if not entidades and not relacionamentos:  # Primeira iteração
            entidades, relacionamentos = processar_questao_pesquisa(questao)

            # Mapear entidades e sugerir novas se necessário
            entidades_mapeadas, entidades_nao_encontradas = MapeamentoEntidades.mapear_entidades(entidades, driver)
            if entidades_nao_encontradas:
                sugestoes_entidades = MapeamentoEntidades.sugerir_novas_entidades(entidades_nao_encontradas)
                print("\nAlgumas entidades não foram encontradas no modelo. Sugestões:")
                for sugestao in sugestoes_entidades:
                    print(f"- {sugestao['entidade_nao_encontrada']}: {', '.join(sugestao['sugestoes'])}")

                # Adicionar as sugestões à lista de entidades (opcional)
                # entidades.extend([sugestao["entidade_nao_encontrada"] for sugestao in sugestoes_entidades])

        else:
            novas_entidades, novos_relacionamentos = processar_questao_pesquisa(questao)
            entidades.extend(novas_entidades)
            relacionamentos.extend(novos_relacionamentos)

        print("\nEntidades identificadas:")
        for entidade in entidades:
            print(f"- {entidade}")

        print("\nRelacionamentos identificados:")
        for relacionamento in relacionamentos:
            print(f"- {relacionamento[0]} ({relacionamento[1]})")

        consulta = gerar_consulta_neo4j(entidades, relacionamentos)
        print("\nConsulta gerada:")
        print(consulta)

        # Executar a consulta no Neo4j e apresentar os resultados
        with driver.session() as session:
            resultado = session.run(consulta)

            # Verificar se há resultados
            if resultado.peek() is None:
                print("\nNenhum resultado encontrado para a consulta.")
            else:
                print("\nResultados da consulta:")
                for registro in resultado:
                    # Apresentar os nós e relacionamentos encontrados
                    for node in registro.values():
                        if isinstance(node, Node):
                            print(f"Nó: {node['nome']} (tipo: {list(node.labels)[0]})")
                        elif isinstance(node, Relationship):
                            print(f"Relacionamento: {node.start_node['nome']} -[{node.type}]-> {node.end_node['nome']}")

        while True:
            resposta = input("\nDeseja explorar mais detalhes sobre alguma entidade ou relacionamento, ou realizar uma busca entre entidades? (s/n): ")
            if resposta.lower() != "s":
                break

            opcao = input("Digite 'detalhar' para explorar uma entidade/relacionamento existente, ou 'buscar' para realizar uma busca entre entidades, ou 'adicionar' para sugerir novas entidades/relacionamentos: ")

            if opcao.lower() == 'detalhar':
                print("Entidades e relacionamentos atuais:")
                for i, entidade in enumerate(entidades):
                    print(f"{i+1}. Entidade: {entidade}")
                for i, relacionamento in enumerate(relacionamentos):
                    print(f"{i+len(entidades)+1}. Relacionamento: {relacionamento[0]} ({relacionamento[1]})")

                indice_elemento = int(input("Digite o número do elemento a ser explorado: ")) - 1

                if indice_elemento < len(entidades):
                    entidade_selecionada = entidades[indice_elemento]
                    # Explosão do subgrafo
                    subgrafo = ExplosaoSubgrafos.recuperar_subgrafo(entidade_selecionada)
                    ExplosaoSubgrafos.apresentar_subgrafo(subgrafo)

                    # Recomendações (opcional, dependendo da sua implementação)
                    recomendacoes = RecomendacaoProjetos.gerar_recomendacoes(entidades, relacionamentos, subgrafo)
                    for recomendacao in recomendacoes:
                        print(f"- {recomendacao['projeto']}")
                        print(f"  Justificativa: {recomendacao['justificativa']}")
                        print(f"  Fontes: {recomendacao['fontes']}\n")

                else:
                    relacionamento_selecionado = relacionamentos[indice_elemento - len(entidades)]
                    # ... (implementação para detalhar um relacionamento, se aplicável)
                    print(f"Relacionamento selecionado:\n {relacionamento_selecionado}")
                    print()

            elif opcao.lower() == 'buscar':
                print("Entidades atuais:")
                for i, entidade in enumerate(entidades):
                    print(f"{i+1}. {entidade}")

                indice_inicio = int(input("Digite o número da entidade de início da busca: ")) - 1
                indice_fim = int(input("Digite o número da entidade de fim da busca: ")) - 1

                entidade_inicio = entidades[indice_inicio]
                entidade_fim = entidades[indice_fim]

                # Consulta para encontrar caminhos entre as entidades
                consulta_caminhos = f"""
                    MATCH path = (inicio:Entidade {{nome: '{entidade_inicio}'}})-[*]-(fim:Entidade {{nome: '{entidade_fim}'}})
                    RETURN path
                """

                with driver.session() as session:
                    resultado_caminhos = session.run(consulta_caminhos)

                    # Apresentar os caminhos encontrados
                    if resultado_caminhos.peek() is None:
                        print("\nNenhum caminho encontrado entre as entidades.")
                    else:
                        print("\nCaminhos encontrados:")
                        for registro in resultado_caminhos:
                            caminho = registro["path"]
                            print(caminho)

            elif opcao.lower() == 'adicionar':
                questao_adicional = input("Digite a questão de pesquisa que as novas entidades/relacionamentos devem abordar: ")
                novas_entidades, novos_relacionamentos = ProcessamentoLinguagemNatural.processar_questao_pesquisa(questao_adicional)

                # Apresentar as sugestões ao usuário para confirmação
                print("\nSugestões de entidades:")
                for entidade in novas_entidades:
                    print(f"- {entidade}")

                print("\nSugestões de relacionamentos:")
                for relacionamento in novos_relacionamentos:
                    print(f"- {relacionamento[0]} ({relacionamento[1]})")

                confirmacao = input("\nDeseja adicionar estas sugestões ao modelo? (s/n): ")
                if confirmacao.lower() == 's':
                    # Persistir as sugestões no Neo4j
                    with driver.session() as session:
                        for entidade in novas_entidades:
                            session.run("""
                                MERGE (e:SugestaoEntidade {nome: $nome})
                                ON CREATE SET e.data_criacao = datetime()
                            """, nome=entidade)

                        for relacionamento in novos_relacionamentos:
                            sujeito, tipo_relacionamento = relacionamento
                            session.run("""
                                MERGE (s:SugestaoEntidade {nome: $sujeito})
                                ON CREATE SET s.data_criacao = datetime()
                                MERGE (r:SugestaoRelacionamento {tipo: $tipo})
                                ON CREATE SET r.data_criacao = datetime()
                                MERGE (s)-[:TEM_SUGESTAO]->(r)
                            """, sujeito=sujeito, tipo=tipo_relacionamento)

                    print("Sugestões de entidades e relacionamentos adicionadas ao banco de dados.")
                else:
                    print("Sugestões descartadas.")

                break 

            else:
                print("Opção inválida. Digite 'detalhar', 'buscar' ou 'adicionar'.")


In [ ]:
# Iniciar a interação
interagir_com_pesquisador()

# Estruturação da Lógica de Detalhamento Sucessivo

## Identificar Entidades e Relacionamentos:

O módulo de processamento de linguagem natural (PLN) continuará sendo utilizado para extrair entidades e relacionamentos das questões de pesquisa dos usuários.
No entanto, precisaremos adaptar as regras de extração para identificar termos e conceitos específicos do domínio de PDI em saúde e do CEIS.
Podemos utilizar técnicas de reconhecimento de entidades nomeadas (NER) e mineração de terminologia para aprimorar a identificação de entidades relevantes.

## Mapear fenômenos no CEIS para o Modelo de Grafo:

As entidades e relacionamentos extraídos pela PLN são mapeados para o modelo de grafo, começando pelo nível de maior abstração.

Quando uma entidade não está presente no modelo atual, o sistema informa o usuário a lacuna nos dados e pode sugerir entidades relacionadas (por similaridade semântica) ou e oferece a possibilidade de incluir a nova entidade no modelo.

Os relacionamentos identificados guiam a navegação pelo grafo, expandindo os subgrafos relevantes para níveis de detalhamento maiores, conforme necessário.

## Detalhar sucessivamente por explosão em Subgrafos:

A lógica de detalhamento sucessivo foi implementada através da explosão de subgrafos.

Quando o usuário deseja mais detalhes sobre uma determinada entidade ou relacionamento, o sistema consulta o Neo4j para recuperar os nós e relacionamentos filhos associados a essa entidade no próximo nível de detalhamento.

Os novos subgrafos são apresentados ao usuário, permitindo que ele explore o modelo em diferentes níveis de granularidade.

## Interagir por meio de Recomendações Contrafactuais Explanáveis:

Para fornecer recomendações diversas, tais como recomendar estratégias para possíveis projetos de PDI, o sistema utiliza algoritmos de análise de grafos e mineração de dados para identificar padrões, tendências e oportunidades no CEIS.

As recomendações são explanáveis, ou seja, o sistema fornece ao usuário as razões e evidências contrafactuais que justificam cada sugestão, com base nos dados do grafo e em fontes externas confiáveis, previamente inseridas por meio de curadoria humana.

A similaridade semântica entre as entidades e relacionamentos da consulta do usuário e os elementos do grafo é utilizada para personalizar as recomendações e garantir sua relevância.

# Classes e Métodos Adicionais:

MapeamentoEntidades: Classe responsável por mapear as entidades extraídas pela PLN para o modelo de grafo e sugerir novas entidades quando necessário.

ExplosaoSubgrafos: Classe responsável por realizar a consulta ao Neo4j para recuperar os subgrafos filhos de uma entidade e apresentá-los ao usuário.

RecomendacaoProjetos: Classe responsável por analisar o grafo e gerar recomendações explanáveis sobre possíveis projetos de PDI, utilizando dados do grafo e fontes externas.

FontesDados: Classe ou módulo para gerenciar as fontes de dados externas (artigos científicos, bases de dados governamentais, etc.